In [1]:
# Установка зависимостей и используемого ПО: 
!git clone https://github.com/AnnaMedved/medicines-classification.git
%cd medicines-classification
!pip install -r bert_model/requirements.txt
!pip install transformers
!git lfs install
!git clone https://huggingface.co/cointegrated/rubert-tiny bert_model/rubert-tiny

Cloning into 'medicines-classification'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 83 (delta 18), reused 28 (delta 14), pack-reused 51
Unpacking objects: 100% (83/83), 97.48 MiB | 5.21 MiB/s, done.
Updating files: 100% (29/29), done.
/content/medicines-classification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 68.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement scikit-learn==0.22.2 (from versions: 0.9, 0.10, 0.11, 0.12, 0

### Загрузка данных и получение результата

In [1]:
# ================================================================
data = 'data.xlsx' # название файла с расширением 
# columns = ['column1', ..., 'columnN'] # все интересуемые столбцы
columns = ['Правило взаимодействия (обр.)', 'Исходники (обр.)']
# =================================================================

import pandas as pd
from bert_model.preprocessing_data import all_preprocessing
df = pd.read_excel(data)
# df, features = all_preprocessing(data, columns)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Анечка\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Анечка\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
df['Подуровень'].value_counts()

2    2829
6    2336
8    2128
4    1117
5    1055
7     331
3     274
1      24
Name: Подуровень, dtype: int64

In [2]:
from bert_model.bert_dataset import CustomDataset
from bert_model.bert_classifier import BertClassifier

num_classes = len(df['Подуровень'].unique())
classifier = BertClassifier(
        model_path='bert_model/rubert-tiny',
        tokenizer_path='bert_model/rubert-tiny',
        n_classes=num_classes,
        epochs=1,
        model_save_path='bert_model/bert_v1.pt'
)

# import torch 
# classifier = torch.load('bert_model/bert_v1.pt')

Some weights of the model checkpoint at bert_model/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not ini

In [4]:
print(features)

['text_sw_0', 'text_sw_1']


In [10]:
import numpy as np 
# predictions = np.zeros((len(df), len(features)))

for num, feature in enumerate(features):   
  to_pred = df.loc[:, feature]
  predictions = [classifier.predict(t) for t in to_pred]
  print(predictions)

[4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 0, 3, 3, 0, 3, 3, 0, 0, 0, 0, 3, 0, 3, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 4, 3, 4, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 4, 4, 0, 0, 4, 0, 0, 4, 4, 4, 0, 3, 0, 4, 4, 0, 4, 0, 4, 0, 4, 4, 0, 0, 4, 4, 4, 4, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4, 0, 0, 4, 4, 4, 0, 0, 3, 8, 6, 6, 5, 6, 0, 3, 3, 3, 3, 4, 4, 4, 0, 4, 4, 4, 6, 0, 0, 3, 3, 4, 4, 0, 0, 0, 3, 0, 0, 4, 4, 3, 0, 3, 3, 6, 3, 0, 3, 0, 4, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 6, 0, 3, 0, 4, 0, 0, 3, 0, 3, 0, 3, 0, 0, 4, 4, 0, 0, 4, 0, 0, 0, 5, 0, 0, 0, 3, 0, 3, 5, 0, 6, 3, 0, 3, 4, 0, 0, 6, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 4, 4, 3, 3, 3, 4, 4, 3, 0, 3, 4, 3, 4, 3, 4, 3, 3, 3, 0, 3, 0, 6, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

### Получение результата

In [ ]:
df.head(15)

,ИД правила,Класс,Подуровень,Дата акт.,ИД источника,Источник,Левое ДВ,Левая ФГ,Правое ДВ,Правая ФГ,Правило взаимодействия (обр.),Исходники (обр.),Дата ИМП,preproccessed,prep_text,text_stem,text_sw
0,38039,2,6,2014-12-22,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,0020 Антиаритмические средства,Верапамил*,0030 Блокаторы кальциевых каналов,Из-за риска развития брадикардии и нарушений п...,• из-за риска развития брадикардии и нарушений...,NaN,• из за риска развития брадикардии и нарушений...,• из за риска развития брадикардии и нарушений...,риск развит брадикард нарушен проводим рекомен...,риска развития брадикардии нарушений проводимо...
1,38041,2,6,2014-12-22,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,0020 Антиаритмические средства,Дигоксин*,0090 Сердечные гликозиды и негликозидные карди...,Из-за риска развития брадикардии и нарушений п...,• из-за риска развития брадикардии и нарушений...,NaN,• из за риска развития брадикардии и нарушений...,• из за риска развития брадикардии и нарушений...,риск развит брадикард нарушен проводим рекомен...,риска развития брадикардии нарушений проводимо...
2,38040,2,6,2019-02-04,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,0020 Антиаритмические средства,Дилтиазем*,0030 Блокаторы кальциевых каналов,Из-за риска развития брадикардии и нарушений п...,• из-за риска развития брадикардии и нарушений...,NaN,• из за риска развития брадикардии и нарушений...,• из за риска развития брадикардии и нарушений...,риск развит брадикард нарушен проводим рекомен...,риска развития брадикардии нарушений проводимо...
3,68429,2,8,2022-11-30,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),7-[N-(4-трифторметилбензоил)-гидразинокарбонил...,0030 Противовирусные средства,Аторвастатин*,0040 Статины,Следует с осторожностью назначать одновременно...,"НИОХ-14 (капс.), 04.10.2022\nСледует с осторож...",04.10.2022,НИОХ капс Следует с осторожностью назначать од...,ниох капс следует с осторожностью назначать од...,ниох капс след осторожн назнача одновремен тек...,ниох капс следует осторожностью назначать одно...
4,68426,2,2,2022-11-30,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),7-[N-(4-трифторметилбензоил)-гидразинокарбонил...,0030 Противовирусные средства,Бупропион*,0020 Антидепрессанты,Совместное применение тековиримата (активный м...,"НИОХ-14 (капс.), 04.10.2022\nСовместно примене...",04.10.2022,НИОХ капс Совместно применение Тековиримата с ...,ниох капс совместно применение тековиримата с ...,ниох капс совместн применен тековиримат мидазо...,ниох капс совместно применение тековиримата ми...
5,68433,2,8,2022-11-30,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),7-[N-(4-трифторметилбензоил)-гидразинокарбонил...,0030 Противовирусные средства,Варденафил*,0026 Ингибиторы ФДЭ-5,Следует с осторожностью назначать одновременно...,"НИОХ-14 (капс.), 04.10.2022\nСледует с осторож...",04.10.2022,НИОХ капс Следует с осторожностью назначать од...,ниох капс следует с осторожностью назначать од...,ниох капс след осторожн назнача одновремен тек...,ниох капс следует осторожностью назначать одно...
6,68422,2,8,2022-11-30,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),7-[N-(4-трифторметилбензоил)-гидразинокарбонил...,0030 Противовирусные средства,Вориконазол*,0050 Противогрибковые средства,Следует с осторожностью назначать одновременно...,"НИОХ-14 (капс.), 04.10.2022\nСледует с осторож...",04.10.2022,НИОХ капс Следует с осторожностью назначать од...,ниох капс следует с осторожностью назначать од...,ниох капс след осторожн назнача одновремен тек...,ниох капс следует осторожностью назначать одно...
7,68434,2,8,2022-11-30,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),7-[N-(4-трифторметилбензоил)-гидразинокарбонил...,0030 Противовирусные средства,Дарунавир*,0010 Средства для лечения ВИЧ-инфекции,Следует с осторожностью назначать одновременно...,"НИОХ-14 (капс.), 04.10.2022\nСледует с о

In [ ]:
import pandas as pd
import numpy as np
import string
import nltk

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from nltk.stem.snowball import SnowballStemmer 
from tqdm.auto import tqdm, trange
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Анна\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Анна\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os 
print(os.getcwd())

d:\Documents\medicines-classification\bert_model


In [ ]:
df = pd.read_excel('data.xlsx')

In [ ]:
df.shape

(10094, 13)

In [ ]:
df.head(5)

,ИД правила,Класс,Подуровень,Дата акт.,ИД источника,Источник,Левое ДВ,Левая ФГ,Правое ДВ,Правая ФГ,Правило взаимодействия (обр.),Исходники (обр.),Дата ИМП
0,38039,2,6,2014-12-22,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,0020 Антиаритмические средства,Верапамил*,0030 Блокаторы кальциевых каналов,Из-за риска развития брадикардии и нарушений п...,• из-за риска развития брадикардии и нарушений...,NaN
1,38041,2,6,2014-12-22,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,0020 Антиаритмические средства,Дигоксин*,0090 Сердечные гликозиды и негликозидные карди...,Из-за риска развития брадикардии и нарушений п...,• из-за риска развития брадикардии и нарушений...,NaN
2,38040,2,6,2019-02-04,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),4-нитро-N-[(1RS)-1-(4-фторфенил)-2-(1-этилпипе...,0020 Антиаритмические средства,Дилтиазем*,0030 Блокаторы кальциевых каналов,Из-за риска развития брадикардии и нарушений п...,• из-за риска развития брадикардии и нарушений...,NaN
3,68429,2,8,2022-11-30,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),7-[N-(4-трифторметилбензоил)-гидразинокарбонил...,0030 Противовирусные средства,Аторвастатин*,0040 Статины,Следует с осторожностью назначать одновременно...,"НИОХ-14 (капс.), 04.10.2022\nСледует с осторож...",04.10.2022
4,68426,2,2,2022-11-30,88.0,1б. Инструкция на ЛС (grls.rosminzdrav.ru),7-[N-(4-трифторметилбензоил)-гидразинокарбонил...,0030 Противовирусные средства,Бупропион*,0020 Антидепрессанты,Совместное применение тековиримата (активный м...,"НИОХ-14 (капс.), 04.10.2022\nСовместно примене...",04.10.2022


In [ ]:
df['Подуровень'].value_counts()

2    2829
6    2336
8    2128
4    1117
5    1055
7     331
3     274
1      24
Name: Подуровень, dtype: int64

In [ ]:
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else '' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else '' for i in text])


cyrillic = [chr(i) for i in range(ord('а'), ord('я') + 1)]
def remove_symbols(text): 
    return ''.join([s if s not in cyrillic else '' for s in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])
russian_stopwords.extend(['•', '…', '«', '»', '...', 'т.д.', 'т', 'д'])

preproccessing = lambda text: (remove_symbols(remove_multiple_spaces(remove_numbers(remove_punctuation(text)))))

df['preproccessed_sources'] = list(map(preproccessing, df['Исходники (обр.)']))
df['preproccessed_rule'] = list(map(preproccessing, df['Правило взаимодействия (обр.)']))

prep_text = [remove_symbols(remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower())))) for text in tqdm(df['Исходники (обр.)'])]
prep_text_2 = [remove_symbols(remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower())))) for text in tqdm(df['Правило взаимодействия (обр.)'])]

df['prep_1'] = prep_text
df['prep_2'] = prep_text_2

# def lemmatize_text(text):
#     tokens = mystem.lemmatize(text.lower())
#     tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
#     text = " ".join(tokens)
#     return text

100%|██████████| 10094/10094 [00:03<00:00, 3083.46it/s]


In [ ]:

# text = df['prep_1'][0]
# word_tokenize(text) 

In [ ]:
stemmer = SnowballStemmer("russian") 
russian_stopwords = stopwords.words("russian")

stemmed_texts_list = []
for text in tqdm(df['prep_1']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

stemmed_texts_list_2 = []
for text in tqdm(df['prep_2']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list_2.append(text)

df['text_1'] = stemmed_texts_list
df['text_2'] = stemmed_texts_list_2

100%|██████████| 10094/10094 [00:01<00:00, 9751.09it/s]


In [ ]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)
  
sw_texts_list = []
for text in tqdm(df['prep_1']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

  
sw_texts_list_2 = []
for text in tqdm(df['prep_2']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list_2.append(text)

df['text_sw_1'] = sw_texts_list
df['text_sw_2'] = sw_texts_list_2


100%|██████████| 10094/10094 [00:00<00:00, 12484.24it/s]


In [ ]:
df['text_sw_1'][0]

'риска развития брадикардии нарушений проводимости рекомендуется введение препарата фоне приема бета адреноблокаторов блокаторов « медленных » кальциевых каналов урежающих чсс верапамил дилтиазем дигоксина'

In [ ]:
df.to_csv('rls_stemmed.csv')

In [ ]:
lemm_texts_list = []
for text in tqdm(df['text_sw_1']):

    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)

lemm_texts_list_2 = []
for text in tqdm(df['text_sw_2']):

    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list_2.append(text)
    except Exception as e:
        print(e)
    
df['text_lemm_1'] = lemm_texts_list
df['text_lemm_2'] = lemm_texts_list

  0%|          | 0/10094 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ']
    return " ".join(tokens)

In [ ]:
df.to_csv('lemm.csv')

In [ ]:
df_lemm = pd.read_csv('lemm.csv', encoding = 'utf-8')

In [ ]:
df_lemm

In [ ]:
X = df_lemm['text_sw']
y = df_lemm['Класс']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
my_tags = df_lemm['Класс'].unique()
my_tags

## BOW with Keras

In [ ]:
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_posts = X_train
train_tags = y_train

test_posts = X_test 
test_tags = y_test

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 100

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

## Наивный байесовский классификатор

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
%%time
nb.fit(X_train, y_train)

In [ ]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

In [ ]:
y_pred[0]

In [ ]:
print(X_test[0], y_test[0], y_pred[0])

In [ ]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

## Метод опорных векторов

In [ ]:
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
%%time
sgd.fit(X_train, y_train)

In [ ]:
%%time
y_pred = sgd.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

## Логистическая регрессия

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [ ]:
%%time
logreg.fit(X_train, y_train)

In [ ]:
%%time
y_pred = logreg.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,targe))